In [1]:
from heat_pump_model_v2 import *
from electric_model import *
from gas_model import *

from libraries import * 
from refrigerant_properties import *
import numpy as np
from utilities.unit_defs import Q_

from cashflow_model import *

In [2]:
hp_test = heat_pump()
hp_test.construct_yaml_input_quantities('heatpump_model_inputs.yml')
hp_test.hot_temperature_desired = Q_(np.array([120]*8760), 'degC')
hp_test.hot_temperature_minimum = Q_(np.array([80]*8760), 'degC')
hp_test.cold_temperature_available = Q_(np.array([60]*8760), 'degC')
hp_test.carnot_efficiency_factor = Q_('0.55')
hp_test.carnot_efficiency_factor_flag = False
hp_test.refrigerant = 'R234a'

hp_test.run_all('hp_test')

print()
#print(round(hp_test.capital_cost,0))
#print(300*1000/hp_test.actual_COP)
print()

Calculate COP Called
Average Theoretical COP:  5.687857142857141 dimensionless
Average Estimated COP:  2.8439285714285707 dimensionless
Calculate Energy and Mass Called
Hot Mass Flow Average: 99.0 kg/s
Cold Average Outlet Temperature: 56.00 °C
Average Power Draw of Heat Pump: 1408.160 kW
Maximum Power Draw of Heat Pump: 1408.160 kW
Annual Electricity in: 12,335,479.8 kWh
Calculate Heat Pump Costs
Capital Cost: 1,201,411.76 $
Capacity Factor: 1.000
One Year Fixed O&M Costs: 161.24 $·MMBtu/h/kW/yr
One Year Variable O&M Costs: 5,985.10 $/yr
One Year Energy Costs: 415,688.77 $/yr
One Year Operating Costs: 1,600.05 $·MMBtu/h/kW/yr
Lifetime LCOH: 4.42 $/MMBtu
Writing all output to a file




In [3]:
heater_test = electric_heater()
heater_test.construct_yaml_input_quantities('electric_model_inputs.yml')
heater_test.hot_temperature = Q_(np.array([120]*8760), 'degC')
heater_test.cold_temperature = Q_(np.array([60]*8760), 'degC')
heater_test.run_all('electric_test')


Calculate Energy and Mass Called
Hot Mass Flow Average: 0.406 kg/s
Average Power Draw of Electric Heater: 1086.957 kW
Maximum Power Draw of Electric Heater: 1086.957 kW
Annual Electricity in: 9,521,739.1 kWh
Capital Cost: 100,000.00 $
Capacity Factor: 1.000
One Year Fixed O&M Costs: 3.41 $·MMBtu/h/kW/yr
One Year Variable O&M Costs: 298.90 $/yr
One Year Energy Costs: 130,456.52 $/yr
One Year Operating Costs: 449.57 $·MMBtu/h/kW/yr
Lifetime LCOH: 4.58 $/MMBtu
Writing all output to a file


In [4]:
gas_test = gas_heater()
gas_test.construct_yaml_input_quantities('gas_model_inputs.yml')
gas_test.hot_temperature = Q_(np.array([120]*8760), 'degC')
gas_test.cold_temperature = Q_(np.array([60]*8760), 'degC')
gas_test.run_all('gas_test')

print(gas_test.capital_cost)

Calculate Energy and Mass Called
Hot Mass Flow Average: 0.406 kg/s
Average Power Draw of Gas Heater: 1250.000 kW
Maximum Power Draw of Gas Heater: 1250.000 kW
Annual Gas in: 36.6 MMSCF
Capital Cost: 3,070.93 $
Capacity Factor: 1.000
One Year Fixed O&M Costs: 170.61 $/yr
One Year Variable O&M Costs: 298.90 $/yr
One Year Energy Costs: 747,258.91 $/yr
One Year Operating Costs: 747,728.42 $/yr
Lifetime LCOH: 25.02 $/MMBtu
Writing all output to a file
3070.9270408395387 USD


In [5]:
## Converting one component to intput dict
gas_dict = object_to_dict(gas_test)
elec_dict = object_to_dict(heater_test)

calculate_cash_flow(gas_dict, elec_dict, 20, 0.10)
#calculate_cash_flow(elec_dict, gas_dict, 20, 0.10)

[-96929.07295916046, 637641.7467599825, 658990.930600906, 681038.993948774, 703808.4918439785, 727322.7006139726, 751605.6408011392, 776682.1008165794, 802577.6613427387, 829318.7205085104, 856932.5198612154, 885447.1711606063, 914891.6840208557, 945295.9944272889, 976690.9941554839, 1009108.561121208, 1042581.5906905859, 1077144.027980792, 1112830.9011825372, 1149678.3559365885]
NPV: 6,553,267.24 $
IRR:  0.06611914134871409 percent
PBP:  -0.15829159369364154


In [6]:
gas_dict = object_to_dict(gas_test)
hp_dict = object_to_dict(hp_test)

calculate_cash_flow(gas_dict, hp_dict, 20, 0.10)

[-1198340.8327070435, 340461.49661555875, 355576.0853697058, 371256.1332959403, 387521.3402394885, 404392.0662314607, 421889.3531091558, 440034.9468343019, 458851.3205315547, 478361.69827028125, 498590.0796134024, 519561.26495780447, 541300.8816916281, 563835.4111945301, 587192.2167078583, 611399.5721025175, 636486.6915732096, 662483.7602886182, 689421.9660280651, 717333.5318361248]
NPV: 2,602,619.80 $
IRR:  0.0032458543619215564 percent
PBP:  -3.164056646163533


In [7]:
gas_dict = object_to_dict(gas_test)
elec_dict = combined_dict(heater_test, hp_test)

calculate_cash_flow(gas_dict, elec_dict, 20, 0.10)

[-1298340.8327070435, 206910.88960474601, 219173.96492497995, 231941.47702981567, 245231.8198853872, 259064.01969789993, 273457.75593589514, 288433.3830376519, 304011.95282577456, 320215.23765171587, 337065.754293718, 354586.7886323972, 372802.42112897546, 391737.5531319551, 411417.93403886247, 431870.18934052344, 453121.84957622725, 475201.38022901875, 498138.2125913085, 521962.7756319401]
NPV: 1,219,295.99 $
IRR:  0.00201341648830496 percent
PBP:  -5.110027829578083
